<a href="https://colab.research.google.com/github/LostRuins/koboldcpp/blob/concedo/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Welcome to the Official KoboldCpp Colab Notebook
It's really easy to get started. Just press the two **Play** buttons below, and then connect to the **Cloudflare URL** shown at the end. 
You can select a model from the dropdown or enter a URL into the Manual URL field to use any GGUF model from Huggingface. Formating for a manual URL should look like `https://huggingface.co/KoboldAI/LLaMA2-13B-Tiefighter-GGUF/resolve/main/LLaMA2-13B-Tiefighter.Q4_K_M.gguf`

In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldCpp below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title <b>v-- Enter your model below and then click this to start Koboldcpp</b>

Model = "Tiefighter 13B Q4_K_M 4k Context" #@param ["Tiefighter 13B Q4_K_M 4k Context","Tiefighter 13B Q4_K_S 6k Context","Mythomax 13B Q4_K_M 4k Context","Mythomax 13B Q4_K_S 6k Context","ReMM-SLERP 13B Q4_K_M 4k Context","ReMM-SLERP 13B Q4_K_S 6k Context","Xwin LM 13B v0.2 Q4_K_M 4k Context","Xwin LM 13B v0.2 Q4_K_S 6k Context","Stheno 13B Q4_K_M 4k Context","Stheno 13B Q4_K_M 4k Context","Stheno 13B Q4_K_S 6k Context","OpenHermes-2-Mistral 7B Q5_K_M 8k Context","Mistral-Claude-Chat 7B Q5_K_M 8k Context"]{allow-input: true}
Manual_Link = False #@param {type:"boolean"}
Manual_Context = False #@param {type:"boolean"}
Model_URL = "OPTIONAL: Place URL Here" #@param [""]{allow-input: true}
ContextSize = 4096 #@param [4096,6144,8192] {allow-input: true}


if Model == "Tiefighter 13B Q4_K_M 4k Context":
  Model = "https://huggingface.co/KoboldAI/LLaMA2-13B-Tiefighter-GGUF/resolve/main/LLaMA2-13B-Tiefighter.Q4_K_M.gguf"
  Context = 4096
if Model == "Tiefighter 13B Q4_K_S 6k Context":
  Model = "https://huggingface.co/KoboldAI/LLaMA2-13B-Tiefighter-GGUF/resolve/main/LLaMA2-13B-Tiefighter.Q4_K_S.gguf"
  Context = 6144
elif Model == "Mythomax 13B Q4_K_M 4k Context":
  Model = "https://huggingface.co/TheBloke/MythoMax-L2-13B-GGUF/resolve/main/mythomax-l2-13b.Q4_K_M.gguf"
  Context = 4096
elif Model == "Mythomax 13B Q4_K_S 6k Context":
  Model = "https://huggingface.co/TheBloke/MythoMax-L2-13B-GGUF/resolve/main/mythomax-l2-13b.Q4_K_S.gguf"
  Context = 6144
elif Model == "ReMM-SLERP 13B Q4_K_M 4k Context":
  Model = "https://huggingface.co/TheBloke/ReMM-SLERP-L2-13B-GGUF/resolve/main/remm-slerp-l2-13b.Q4_K_M.gguf"
  Context = 4096
elif Model == "ReMM-SLERP 13B Q4_K_S 6k Context":
  Model = "https://huggingface.co/TheBloke/ReMM-SLERP-L2-13B-GGUF/resolve/main/remm-slerp-l2-13b.Q4_K_S.gguf"
  Context = 6144
elif Model == "Xwin LM 13B v0.2 Q4_K_M 4k Context":
  Model = "https://huggingface.co/TheBloke/Xwin-LM-13B-v0.2-GGUF/resolve/main/xwin-lm-13b-v0.2.Q4_K_M.gguf"
  Context = 4096
elif Model == "Xwin LM 13B v0.2 Q4_K_S 6k Context":
  Model = "https://huggingface.co/TheBloke/Xwin-LM-13B-v0.2-GGUF/resolve/main/xwin-lm-13b-v0.2.Q4_K_S.gguf"
  Context = 6144
elif Model == "Stheno 13B Q4_K_M 4k Context":
  Model = "https://huggingface.co/TheBloke/Stheno-L2-13B-GGUF/resolve/main/stheno-l2-13b.Q4_K_M.gguf"
  Context = 4096
elif Model == "Stheno 13B Q4_K_S 6k Context":
  Model = "https://huggingface.co/TheBloke/Stheno-L2-13B-GGUF/resolve/main/stheno-l2-13b.Q4_K_S.gguf"
  Context = 6144
elif Model == "OpenHermes-2-Mistral 7B Q5_K_M 8k Context":
  Model = "https://huggingface.co/TheBloke/OpenHermes-2-Mistral-7B-GGUF/resolve/main/openhermes-2-mistral-7b.Q5_K_M.gguf"
  Context = 8192
elif Model == "Mistral-Claude-Chat 7B Q5_K_M 8k Context":
  Model = "https://huggingface.co/TheBloke/Mistral-7B-Claude-Chat-GGUF/resolve/main/mistral-7b-claude-chat.Q5_K_M.gguf"
  Context = 8192
if Manual_Link:
  Model = $Model_URL
if Manual_Context:
  Context = $ContextSize

%cd /content
!git clone https://github.com/LostRuins/koboldcpp
%cd /content/koboldcpp
kvers = !(cat koboldcpp.py | grep 'KcppVersion = ' | cut -d '"' -f2)
kvers = kvers[0]
!echo Finding prebuilt binary for {kvers}
!wget -c https://huggingface.co/concedo/koboldcpp/resolve/main/prebuilt_binaries/{kvers}.so
!test -f {kvers}.so && mv {kvers}.so koboldcpp_cublas.so || echo Prebuilt Binary Does Not Exist
!test -f koboldcpp_cublas.so && echo Prebuilt Binary Exists || make koboldcpp_cublas LLAMA_CUBLAS=1
!cp koboldcpp_cublas.so koboldcpp_cublas.dat
!wget $Model -O model.ggml
!wget -c https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!nohup ./cloudflared-linux-amd64 tunnel --url http://localhost:5001 &
!sleep 8
!cat nohup.out
!python koboldcpp.py model.ggml --usecublas 0 --multiuser --gpulayers 99 --contextsize $Context --hordeconfig concedo 1 1 --onready "echo Connect to the link below && cat nohup.out | grep trycloudflare.com && rm nohup.out"
